In [1]:
import tensorflow as tf
import pathlib
import os
import shutil

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

TensorFlow Version: 2.16.2
Keras Version: 3.8.0


In [2]:
trained_keras_model_filename = 'modelv4.1.keras'
export_dir = 'temp_dir'
tflite_model_filename = 'modelv4.1.tflite'

IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 6
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

In [3]:
keras_model_path = pathlib.Path(trained_keras_model_filename)
full_trained_model = tf.keras.models.load_model(keras_model_path, compile=False)

2025-04-09 18:55:09.055956: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-04-09 18:55:09.055998: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-04-09 18:55:09.056004: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-04-09 18:55:09.056026: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-09 18:55:09.056043: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
base_model_inference = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights=None)

inputs_inference = tf.keras.Input(shape=IMG_SHAPE, name="input_layer")

x_inf = preprocess_input(inputs_inference)
x_inf = base_model_inference(x_inf, training=False) 

In [5]:
x_inf = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x_inf)
x_inf = tf.keras.layers.Dropout(0.3, name="dropout")(x_inf)
outputs_inference = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name="predictions")(x_inf)
inference_model = tf.keras.Model(inputs_inference, outputs_inference, name="squirrel_inference_model")

In [6]:
inference_model.summary()

Model: "squirrel_inference_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ avg_pool                        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 6)              │         7,686 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,265,670 (8.64 MB)

 Trainable params: 2,231,558 (8.51 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [7]:
inference_model.set_weights(full_trained_model.get_weights())

In [8]:
export_path = pathlib.Path(export_dir)
inference_model.export(str(export_path))

INFO:tensorflow:Assets written to: temp_dir/assets


INFO:tensorflow:Assets written to: temp_dir/assets


Saved artifact at 'temp_dir'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  14530046352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530048848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530049232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530049040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530048080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530050384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530050768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530051152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530050960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530044816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14530049616: TensorSpec(shape=(), dtype=tf.resource, name=None)

In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(export_path))

In [10]:
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Оптимизатор (?)

In [11]:
tflite_model = None
tflite_model = converter.convert()

W0000 00:00:1744214197.693206 7837087 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1744214197.693220 7837087 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2025-04-09 18:56:37.693766: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: temp_dir
2025-04-09 18:56:37.699837: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-04-09 18:56:37.699848: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: temp_dir
2025-04-09 18:56:37.804465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-04-09 18:56:37.817727: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2025-04-09 18:56:38.230834: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: temp_dir
2025-04-09 18:56:38.346776: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load fo

In [12]:
shutil.rmtree(export_path)
print(f"Временная папка экспорта удалена: {export_dir}")

Временная папка экспорта удалена: temp_dir


In [13]:
if tflite_model:
    tflite_model_path = pathlib.Path(tflite_model_filename)
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Модель TFLite успешно сохранена в файл: {tflite_model_path}")
    file_size = tflite_model_path.stat().st_size / (1024 * 1024)
    print(f"Размер файла TFLite: {file_size:.2f} MB")

Модель TFLite успешно сохранена в файл: modelv4.1.tflite
Размер файла TFLite: 2.40 MB
